# Convert ONNX Model To Tensorflow Lite

This script is meant to be used in google colab. The following packages must be installed.

In [ ]:
# onnx-tf was designed for TensorFlow 1.X, so force this version.
%tensorflow_version 2.X
!pip install tensorflow
# "!" allows command-line input. Use pip package manager to install the 
# conversion package
!pip install onnx-tf

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 226 kB 6.5 MB/s 
     |████████████████████████████████| 1.1 MB 38.0 MB/s 
     |████████████████████████████████| 13.1 MB 7.0 MB/s 


You need to upload the policy.onnx file created using the matlab script that converts your agent into a onnx file. The model can be uploaded with the upload sign at the left of this window.

In [ ]:
# Set path variables
onnx_path = 'policy.onnx'
#img_zip_path = 'test_images.zip'

# Check that correct files have been uploaded
import os

assert os.path.exists(onnx_path)
#assert os.path.exists(img_zip_path) 

print("Files uploaded successfully.")

import tensorflow as tf
import onnx
from onnx_tf.backend import prepare

onnx_model = 'policy.onnx'

onnx_model = onnx.load('policy.onnx')
tf_rep = prepare(onnx_model)


import os
pb_path = "policy"
tf_rep.export_graph(pb_path)

assert os.path.exists(pb_path)
print(".pb model converted successfully.")

# converter = tf.lite.TFLiteConverter.from_frozen_graph(pb_path,
#                                                       input_arrays=input_nodes,
#                                                       output_arrays=output_nodes)
converter = tf.compat.v1.lite.TFLiteConverter.from_saved_model(pb_path)

tflite_rep = converter.convert()

tflite_path = "policy.tflite"
open(tflite_path, "wb").write(tflite_rep)

assert os.path.exists(tflite_path)
print(".tflite model converted successfully.")

Files uploaded successfully.


.pb model converted successfully.
.tflite model converted successfully.


# Test Converted Model

In [ ]:
tflite_path = "policy.tflite"
interpreter = tf.lite.Interpreter(tflite_path)

# Next, memory is allocated for the input and output tensors
interpreter.allocate_tensors()


# Following this, the index of the model's input and output nodes are extracted from the interpreter. These are used to feed input images into the model, and save likelihoods from the output nod
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_index = input_details[0]['index']
output_index = output_details[0]['index']


import numpy as np
action_list = [-0.03, -0.02, -0.01, 0, 0.01, 0.02, 0.03]
results = []
y_likelihoods_tflite, y_labels_tflite = [], []

for i in action_list:
  x1 = np.float32(np.array([[i]])) 
  #x2 = tf.expand_dims([0.0,0.0,0.0,0.0], 0)
  x2 = np.float32([[0,0,0]])

  # Explicitly set input tensor
  interpreter.set_tensor(input_details[0]['index'], x1)
  interpreter.set_tensor(input_details[1]['index'], x2)


  # Free up numpy reference to internal tensor, then invoke
  interpreter.invoke()

  # Explicitly get the value stored within the output tensor
  y_softmax_tflite = interpreter.get_tensor(output_index)

  y_likelihoods_tflite.extend(y_softmax_tflite)

  results.append(y_softmax_tflite)
print(results)
deltaD = action_list[np.argmax(results)]

print('Action Selected {}'.format(deltaD))

[array([[7.998743]], dtype=float32), array([[7.9827375]], dtype=float32), array([[7.9667315]], dtype=float32), array([[7.9507265]], dtype=float32), array([[7.9343505]], dtype=float32), array([[7.917937]], dtype=float32), array([[7.783197]], dtype=float32)]
-0.03


After the convertion is made, you can download the tflite model created. It should appear in the colab folder to your left.